<a href="https://colab.research.google.com/github/sezinbiner/ClassificationOfCovidXray/blob/main/withKfoldLayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
generator_dir = "/content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset"

class_names = ['COVID', 'Normal']
generator = datagen.flow_from_directory(
        generator_dir,
        target_size=(224, 224),  
        batch_size=20,
        shuffle=True,
        class_mode='binary')


Found 13808 images belonging to 2 classes.


In [3]:
def extract_features(number_of_images, generator, batch_size):
  features = np.zeros(shape=(number_of_images, extraction_layer.output_shape[1], extraction_layer.output_shape[2], extraction_layer.output_shape[3]))
  labels = np.zeros(shape=(number_of_images))
  i = 0
  print("loop")
  for inputs_batch, labels_batch in generator:
      print(i)
      features_batch = extraction_layer.predict(inputs_batch)
      features[i * batch_size : (i + 1) * batch_size] = features_batch
      labels[i * batch_size : (i + 1) * batch_size] = labels_batch
      i += 1
      if i * batch_size >= number_of_images:
          break
  print("loop over")
  return features, labels

In [4]:
from keras.preprocessing import image
from tensorflow.python.keras.applications.resnet_v2 import ResNet50V2
import numpy as np
from keras.models import Model

extraction = ResNet50V2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

In [5]:
extraction_layer = Model(extraction.input, extraction.layers[-5].output)
print(extraction_layer.output_shape)

(None, 7, 7, 512)


In [6]:
features,labels = extract_features(13808, generator, generator.batch_size)

loop
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [7]:
from keras.models import Sequential
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer= "adam" ,
    metrics=['accuracy']
)

In [8]:
from sklearn.model_selection import KFold                    
kf = KFold(n_splits=5)
X = features
y = labels

In [9]:
index_list = []
test_index_list = []
#Trying different train and test sets for higher accuracy
for train_index,test_index in kf.split(X):
    index_list.append(train_index)
    index_list.append(test_index)

In [10]:
index_list

[array([ 2762,  2763,  2764, ..., 13805, 13806, 13807]),
 array([   0,    1,    2, ..., 2759, 2760, 2761]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([2762, 2763, 2764, ..., 5521, 5522, 5523]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([5524, 5525, 5526, ..., 8283, 8284, 8285]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([ 8286,  8287,  8288, ..., 11044, 11045, 11046]),
 array([    0,     1,     2, ..., 11044, 11045, 11046]),
 array([11047, 11048, 11049, ..., 13805, 13806, 13807])]

In [11]:
X_train = X[index_list[0]]                           
X_test = X[index_list[1]]
y_train = y[index_list[0]]
y_test = y[index_list[1]]
print("Train:",index_list[0], "Length:", len(y_train),  "Test:", index_list[1], "Length:", len(y_test))

Train: [ 2762  2763  2764 ... 13805 13806 13807] Length: 11046 Test: [   0    1    2 ... 2759 2760 2761] Length: 2762


In [13]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=20,
                    validation_split=0.2)
from sklearn import metrics
y_pred = model.predict(X_test)
probas = np.array(y_pred)
labels = []
for prob in probas:
  if prob < 0.5:
    labels.append(0)
  else:
    labels.append(1)
y_true_labels = y_test
y_pred_labels = labels
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_true_labels, y_pred_labels,pos_label='positive', average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true_labels, y_pred_labels, average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
print('F1 score: %f' % f1)
from sklearn.metrics import classification_report
print(classification_report(y_true_labels, y_pred_labels, target_names=["covid", "normal"]))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true_labels, y_pred_labels))

Epoch 1/10
442/442 [==============================] - 2s 5ms/step - loss: 0.0944 - accuracy: 0.9578 - val_loss: 0.1537 - val_accuracy: 0.9611
Epoch 2/10
442/442 [==============================] - 2s 4ms/step - loss: 0.0895 - accuracy: 0.9636 - val_loss: 0.1758 - val_accuracy: 0.9606
Epoch 3/10
442/442 [==============================] - 2s 4ms/step - loss: 0.0701 - accuracy: 0.9693 - val_loss: 0.1464 - val_accuracy: 0.9638
Epoch 4/10
442/442 [==============================] - 2s 4ms/step - loss: 0.0870 - accuracy: 0.9597 - val_loss: 0.1409 - val_accuracy: 0.9629
Epoch 5/10
442/442 [==============================] - 2s 4ms/step - loss: 0.0726 - accuracy: 0.9681 - val_loss: 0.1550 - val_accuracy: 0.9670
Epoch 6/10
442/442 [==============================] - 2s 4ms/step - loss: 0.0780 - accuracy: 0.9639 - val_loss: 0.1285 - val_accuracy: 0.9611
Epoch 7/10
442/442 [==============================] - 2s 4ms/step - loss: 0.0820 - accuracy: 0.9633 - val_loss: 0.1756 - val_accuracy: 0.9656
Epoch 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
